<a href="https://colab.research.google.com/github/ashwinsapre/GANfaces_iCarl/blob/main/iCarl_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
import keras.layers as L
from keras.datasets import fashion_mnist
from keras.applications import DenseNet121
from keras.applications import VGG16
import matplotlib.pyplot as plt

 For multi-task learning, branching NN out
- one branch predicts the class of FashionMNIST object (classification)
- other branch predicts whether the object is a "top" or not (0/1 output)

In [12]:
def create_model(n_classes, input_dim, cl_weight, b_weight, lr):
    '''
        Creating categorical classification model
    '''
    
    inputs=L.Input((input_dim,input_dim,1))
    
    xc=L.Conv2D(64, kernel_size=3, padding='same', strides=1)(inputs)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.Conv2D(64, kernel_size=3, padding='same', strides=1)(xc)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.MaxPool2D(pool_size=2, strides=2)(xc)
    
    xc=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(inputs)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(xc)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.MaxPool2D(pool_size=2, strides=2)(xc)
    
    xc=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(xc)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(xc)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.MaxPool2D(pool_size=2, strides=2)(xc)
    
    xc=L.Flatten()(xc)
    xc=L.Dense(n_classes)(xc)
    outputc=L.Softmax(name='outputc')(xc)
    
    '''
        Creating binary classification model (a top/not a top)
    '''
    
    x=L.LeakyReLU(0.2)(inputs)
    x=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    x=L.Flatten()(x)
    outputb=L.Dense(1, activation='sigmoid', name='outputb')(x)
    #outputb=L.Activation('sigmoid', name="outputb")(x)

    '''
        Combining both models
    '''
    model=keras.Model(inputs=inputs, outputs=[outputc, outputb])
    losses={'outputc':'categorical_crossentropy', 'outputb':'binary_crossentropy'}
    loss_weights=[cl_weight, b_weight]
    model.compile(optimizer=keras.optimizers.Adam(lr), 
                      loss=losses,
                      loss_weights=loss_weights,
                      metrics=['accuracy'])
    
    return model

In [13]:
def train_model(model, n_epochs:int, x_train, y_trainc, y_trainb, validation_split):
    history=model.fit(x=x_train, 
                      y={'outputc':y_trainc, 'outputb': y_trainb}, 
                      epochs=n_epochs, 
                      validation_split=0.1,
                      shuffle=True)
    return model

In [26]:
def calc_feature_vectors(model, images):
    #extract feature vector from a layer of ALREADY-TRAINED model
    #size of feature vector=784
    feature_model=keras.Model(inputs=model.inputs, outputs=model.get_layer("flatten").output)
    feature_vectors=feature_model.predict(images)
    norms=np.linalg.norm(feature_vectors, keepdims=True, axis=1)
    feature_vectors=feature_vectors/norms
    return feature_vectors

In [15]:
def create_exemplar_set(mem_size, n_classes, feature_vectors, labels, reconstruct=False):
    per_class=mem_size/n_classes

    class_vectors={}
    mean_class_vectors={}
    class_vectors_distances={}

    for i in range(n_classes):
      class_vectors[i]=[]
      mean_class_vectors[i]=[]
      class_vectors_distances[i]=[]
    
    for i in range(len(labels)):
      class_vectors[labels[i]].append(feature_vectors[i])

    for i in range(n_classes):
      mean_class_vectors[i]=np.sum(class_vectors[i], axis=0)/len(class_vectors[i])
    
    #calculating distances from mean
    for i in range(len(labels)):
      class_vectors_distances[labels[i]].append(np.linalg.norm(mean_class_vectors[labels[i]] - feature_vectors[i]))

    return None

In [16]:
def classify(X, mean_exemplar_sets):    
    image_vector=calc_feature_vectors(X)
    #exemplar set closest to X feature map is chosen
    return None

In [17]:
def transform_outputs(y_train, y_test):
    labeldict={}
    labeldict[0]=1
    labeldict[1]=0
    labeldict[2]=1
    labeldict[3]=1
    labeldict[4]=1
    labeldict[5]=0
    labeldict[6]=1
    labeldict[7]=0
    labeldict[8]=0
    labeldict[9]=0
    
    y_trainb=[]
    y_testb=[]
    y_trainc=keras.utils.to_categorical(y_train)
    y_testc=keras.utils.to_categorical(y_test)
    
    for i in range(len(y_train)):
        y_trainb.append(labeldict[y_train[i]])
        
    for i in range(len(y_test)):
        y_testb.append(labeldict[y_test[i]])
    
    return y_trainc, y_testc, np.asarray(y_trainb), np.asarray(y_testb)

In [18]:

(x_train, y_train), (x_test, y_test)=tf.keras.datasets.fashion_mnist.load_data()
x_train=x_train.reshape((x_train.shape[0], 28,28,1))
x_test=x_test.reshape((x_test.shape[0],28,28,1))
y_trainc, y_testc, y_trainb, y_testb=transform_outputs(y_train, y_test)
x_train=x_train/255.0
x_test=x_test/255.0

#np.savetxt("ytrain_transformed.csv", y_train, delimiter=',')
#np.savetxt("ytest_transformed.csv", y_test, delimiter=',')
    
#y_train=np.loadtxt(open("ytrain_transformed.csv", "rb"), delimiter=',')
#y_test=np.loadtxt(open("ytest_transformed.csv", "rb"), delimiter=',')
    
model=create_model(n_classes=10, input_dim=28, cl_weight=1.0, b_weight=1.5, lr=0.0005)
    
train_model(model, 50, x_train, y_trainc, y_trainb, validation_split=0.1)
model.save("trainedmodel.h5")
#-----------------------------------------------------------------------#

Epoch 1/50
1688/1688 [==============================] - 32s 15ms/step - loss: 0.9918 - outputc_loss: 0.7491 - outputb_loss: 0.1618 - outputc_accuracy: 0.7280 - outputb_accuracy: 0.9427 - val_loss: 0.4422 - val_outputc_loss: 0.3754 - val_outputb_loss: 0.0445 - val_outputc_accuracy: 0.8613 - val_outputb_accuracy: 0.9842
Epoch 2/50
1688/1688 [==============================] - 24s 14ms/step - loss: 0.4286 - outputc_loss: 0.3627 - outputb_loss: 0.0439 - outputc_accuracy: 0.8678 - outputb_accuracy: 0.9868 - val_loss: 0.3689 - val_outputc_loss: 0.3167 - val_outputb_loss: 0.0348 - val_outputc_accuracy: 0.8850 - val_outputb_accuracy: 0.9890
Epoch 3/50
1688/1688 [==============================] - 24s 14ms/step - loss: 0.3581 - outputc_loss: 0.3047 - outputb_loss: 0.0356 - outputc_accuracy: 0.8891 - outputb_accuracy: 0.9895 - val_loss: 0.3235 - val_outputc_loss: 0.2806 - val_outputb_loss: 0.0286 - val_outputc_accuracy: 0.9008 - val_outputb_accuracy: 0.9897
Epoch 4/50
1688/1688 [==================

In [19]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [20]:
model.save("trainedmodel", "/content/gdrive/MyDrive/iCarlModel")

INFO:tensorflow:Assets written to: trainedmodel/assets


In [21]:
#model=keras.models.load_model("trainedmodel")

In [27]:
feature_model=keras.Model(inputs=model.inputs, outputs=model.get_layer("flatten").output)
feature_vectors=feature_model.predict(x_test)
norms=np.linalg.norm(feature_vectors, keepdims=True, axis=1)
feature_vectors=feature_vectors/norms

In [36]:
class_vectors={}
mean_class_vectors={}
class_vectors_distances={}

for i in range(10):
  class_vectors[i]=[]
  mean_class_vectors[i]=[]
  class_vectors_distances[i]=[]
    
for i in range(len(y_test)):
  class_vectors[y_test[i]].append(feature_vectors[i])

for i in range(10):
  mean_class_vectors[i]=np.sum(class_vectors[i], axis=0)/len(class_vectors[i])
    
#calculating distances from mean
for i in range(len(y_test)):
  class_vectors_distances[y_test[i]].append(np.linalg.norm(mean_class_vectors[y_test[i]] - feature_vectors[i]))

print(max(class_vectors_distances[0]))

1.1190995
